In [9]:
import pandas as pd
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt
import os

# Class & Function Definitions

#### 1. Graphs

In [10]:
class Vertex:
    def __init__(self, id_=""):
        self.id_ = id_
        self.neighbours = {}
    
    def add_neighbour(self, nbr_vertex, weight=0):
        self.neighbours[nbr_vertex] = weight
    
    def get_vertex_neighbours(self):
        return self.neighbours.keys()
    
    def get_weight(self, neighbour):
        if neighbour in self.neighbours.keys():
            return self.neighbours[neighbour]
        return None
    
    def __eq__(self, other):
        return self.id_ == other.id_

    def __lt__(self, other):
        return self.id_ < other.id_

    def __hash__(self):
        return hash(self.id_)
    
    def __str__(self):
        my_string = f"Vertex {self.id_} is connected to: "
        my_neighbours = [vertex.id_ for vertex in self.get_vertex_neighbours()]
        my_string += ", ".join(my_neighbours)
        return my_string
    
class Graph:
    def __init__(self):
        self.vertices = {}
        
    def _create_vertex(self, id_):
        return Vertex(id_)

    def add_vertex(self, id_):
        self.vertices[id_] = self._create_vertex(id_)

    def get_vertex(self, id_):
        return self.vertices.get(id_, None)
        
    def add_edge(self, start_v, end_v, weight=0):
        if start_v not in self.vertices: 
            self.add_vertex(start_v)
        if end_v not in self.vertices: 
            self.add_vertex(end_v)
        v_start = self.vertices[start_v]
        v_end = self.vertices[end_v]
        v_start.add_neighbour(v_end)

    def get_neighbours(self, id_):
        vertex = self.get_vertex(id_)
        if vertex:
            return [neighbour.id_ for neighbour in vertex.get_vertex_neighbours()]
        return []
 
    def __contains__(self, id_):
        return id_ in self.vertices
    
    def __iter__(self):
        return iter(self.vertices.values())

    @property        
    def num_vertices(self):
        return len(self.vertices)

In [11]:
# def generate_probabilities(D=0.5, n=10):
#     """ 
#     Generates the probabilities of arc (i,j) existing for all i, j in n where i =/= j 
#     Probability values are generated based on (Hall & Posner, 2001)
#     Inputs: 
#         - D          : float, value between 0 and 1 which sets the density of the graph
#         - n          : int, number of nodes to be generated
#     Output: 
#         Returns p_ij, a dictionary which stores the probability values using the following data structure: 
#             p_ij = { i: {j: p_ij} }
#     """
#     p_ij = {}
#     n = int(n)
#     for i in range(1, n+1):
#         p_ij[f"J.{i}"] = {}
#         for j in range(1, n+1):
#             if j != i: 
#                 p_ij[f"J.{i}"][f"J.{j}"] = D*(1-D)**(j-i-1) / (1 - D * (1 - (1-D)**(j-i-1) ))
#     return p_ij 
import decimal


decimal.getcontext().prec = 100

def generate_probabilities(D=0.5, n=10):
    """ 
    Generates the probabilities of arc (i,j) existing for all i, j in n where i =/= j 
    Probability values are generated based on (Hall & Posner, 2001)
    Inputs: 
        - D          : float, value between 0 and 1 which sets the density of the graph
        - n          : int, number of nodes to be generated
    Output: 
        Returns p_ij, a dictionary which stores the probability values using the following data structure: 
            p_ij = { i: {j: p_ij} }
    """
    p_ij = {}
    n = int(n)
    D = decimal.Decimal(D)
    one = decimal.Decimal(1)
    
    for i in range(1, n + 1):
        p_ij[f"J.{i}"] = {}
        for j in range(1, n + 1):
            if j != i:
                exponent = decimal.Decimal(j - i - 1)
                numerator = D * (one - D) ** exponent
                denominator = one - D * (one - (one - D) ** exponent)
                p_ij[f"J.{i}"][f"J.{j}"] = float(numerator / denominator)
    return p_ij 


def generate_graph(n, p_ij, dict_graph={}):
    """ 
    Generates a new graph with arcs and nodes that represent a BOM network. 
    The BOM network is generated based on a given set of probability values (Hall & Posner, 2001)
    Inputs: 
        - n          : int, number of nodes in the graph 
        - p_ij       : dict, probability of arc (i,j) existing (p_ij = { i: {j: p_ij}})
        - dict_graph : dict, alternative method to store graph information using adjacency matrix 
    Output: 
        - g          : graph object containing node and edges information
        - dict_graph : no output, but dict_graph is processed in place 
    Specific Features:
        - The graph generated does not contain self-connecting cycles
    """

    g = Graph()
    edges = []

    for i in range(1, n+1):
        dict_graph[f"J.{i}"] = []
        for j in range(i + 1, n + 1):
            dict_graph[f"J.{j}"] = []
            if p_ij[f"J.{i}"][f"J.{j}"] <= random.random():
                edges.append((f"J.{i}", f"J.{j}"))

    for u, v in edges:
        u, v = str(u), str(v)
        if g.get_neighbours(u):
            continue  # Skip if u already has a successor
        if u < v:
            if not any(g.get_vertex(str(w)) for w in g.get_neighbours(v) if g.get_vertex(u) in g.get_neighbours(w)):
                g.add_edge(u, v)
                dict_graph[u].append(v)
        else:
            if not any(g.get_vertex(str(w)) for w in g.get_neighbours(u) if g.get_vertex(v) in g.get_neighbours(w)):
                g.add_edge(u, v)
                dict_graph[u].append(v)
    return g



def visualize_graph(graph, figure_size=(8, 6), font_type="Century Gothic", font_size=15, node_size=1500, node_color='lightblue', edge_color='black', alpha=1, folder_path=None, file_name=None):
    """ 
    Generates a visualization for a graph object. The visualization is made in such a way that: 
        (1) Nodes with no immediate predecessors are placed to the left of the image
        (2) Nodes with no immediate successors are placed to the right of the image 
        (3) All other remaining nodes are placed in between. 
    Inputs (Required): 
        - graph         : graph object, containing the nodes and edges to be visualized
    Inputs (Optional):
        - figure_size   : tuple, sets the preferred size of the image
        - font_type     : str, sets the font family
        - font_size     : float, sets the size of the font
        - node_size     : float, sets the size of the nodes
        - node_color    : str, sets the color of the nodes
        - edge_color    : str, sets the color of the edges
        - edge_alpha    : float, value between 0 and 1 which sets the density of the edges
    Output: 
        - Visualization of the graph using plt.show()
    """
    G = nx.DiGraph()
    
    for vertex in graph:
        G.add_node(vertex.id_)
        for neighbour in vertex.get_vertex_neighbours():
            G.add_edge(vertex.id_, neighbour.id_)
    
    # Create custom positions for nodes
    pos = {}
    nodes_without_predecessors = [node for node in G.nodes if G.in_degree(node) == 0]
    nodes_without_successors = [node for node in G.nodes if G.out_degree(node) == 0]
    other_nodes = [node for node in G.nodes if node not in nodes_without_predecessors and node not in nodes_without_successors]
    
    # Assign layers
    layers = {node: 0 for node in nodes_without_predecessors}
    for node in other_nodes:
        predecessors = G.predecessors(node)
        layers[node] = max(layers[pred] + 1 for pred in predecessors if pred in layers)
    for node in nodes_without_successors:
        predecessors = G.predecessors(node)
        layers[node] = max(layers[pred] + 1 for pred in predecessors if pred in layers)
    
    # Add subset attribute to nodes
    for node, layer in layers.items():
        G.nodes[node]['subset'] = layer
    
    # Assign positions using multipartite_layout
    pos = nx.multipartite_layout(G, subset_key='subset')
    
    # Draw the graph with custom positions
    plt.figure(figsize=figure_size)
    nx.draw(G, pos, with_labels=True, font_family=font_type, font_weight='bold', font_size=font_size,
            node_color=node_color, node_size=node_size,  
            edge_color=edge_color, alpha=alpha)
    if folder_path and file_name:
        plt.savefig(f"{folder_path}//{file_name}.png")
        # plt.show()
        plt.close()
    # else:
    #     plt.show()

#### 2. Machine Speeds

In [12]:
class LogarithmicDistribution:
    def __init__(self, p):
        if not (0 < p < 1):
            raise ValueError("Parameter p must be between 0 and 1.")
        self.p = p

    def pmf(self, k):
        """Probability mass function of the logarithmic distribution."""
        if k < 1:
            return 0
        return -self.p**k / (k * np.log(1 - self.p))

    def sample(self, size=1):
        """Generate random samples from the logarithmic distribution."""
        # Using inverse transform sampling
        samples = []
        for _ in range(size):
            u = np.random.uniform(0, 1)
            k = 1
            sum_pmf = self.pmf(k)
            while u > sum_pmf:
                k += 1
                sum_pmf += self.pmf(k)
            samples.append(k)
        return np.array(samples)

    def plot_pmf(self, max_k=20):
        """Plot the probability mass function."""
        ks = np.arange(1, max_k + 1)
        pmf_values = [self.pmf(k) for k in ks]
        plt.stem(ks, pmf_values, use_line_collection=True)
        plt.xlabel('k')
        plt.ylabel('P(X=k)')
        plt.title('Logarithmic Distribution PMF')
        plt.show()

In [13]:
def plot_histogram(data, bins=10, title='Histogram', xlabel='Value', ylabel='Frequency'):
    """
    Plots a histogram of the given data.

    Parameters:
    - data: array-like, the input array of numbers.
    - bins: int, the number of bins for the histogram.
    - title: str, the title of the plot.
    - xlabel: str, the label for the x-axis.
    - ylabel: str, the label for the y-axis.
    """
    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=bins, edgecolor='black', alpha=0.7)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()


#### 3. Machine Routing

In [14]:
def generate_transition_matrix(M, bottleneck_machine=None, bottleneck_probability=0.3):
    """
    Generate a transition matrix for machine assignments.
    Inputs:
        - M                         : int, number of machines
        - bottleneck_machine        : int or None, index of the bottleneck machine (0-indexed)
        - bottleneck_probability    : float, probability that the bottleneck machine will be chosen
    Output:
        - transition_matrix         : numpy array of shape (M, M) containing transition probabilities
    """
    transition_matrix = np.ones((M, M))
    if bottleneck_machine is not None:
        bottleneck_machine -= 1  # Convert to 0-indexed
        transition_matrix[:, bottleneck_machine] = bottleneck_probability
        other_probability = (1 - bottleneck_probability) / (M - 1)
        for i in range(M):
            if i != bottleneck_machine:
                transition_matrix[:, i] = other_probability
    transition_matrix = transition_matrix / transition_matrix.sum(axis=1)[:, np.newaxis]
    return transition_matrix

def assign_machines(df, M, bottleneck_machine=None, bottleneck_probability=0.3):
    """
    Assign machines to operations based on a transition probability matrix.
    Inputs:
        - df                        : DataFrame, containing the operations and their predecessors
        - M                         : int, number of machines
        - bottleneck_machine        : int or None, index of the bottleneck machine (0-indexed)
        - bottleneck_probability    : float, probability that the bottleneck machine will be chosen
    Output:
        - df                        : DataFrame, with an additional column "machine" assigned
    """
    transition_matrix = generate_transition_matrix(M, bottleneck_machine, bottleneck_probability)
    current_machine = random.randint(0, M - 1)  # Start with a random machine (0-indexed)

    machine_assignments = []
    for _ in range(len(df)):
        next_machine_idx = np.random.choice(M, p=transition_matrix[current_machine])
        next_machine = f"M{1 + np.random.choice(M, p=transition_matrix[current_machine])}"
        machine_assignments.append(next_machine) 
        current_machine = next_machine_idx
    
    df['machine'] = machine_assignments
    return df

# Data Generation

In [15]:
def generate_dataset(n, M, D, p, folder_path=None, file_name=None, bottleneck_machine=None, display_graph=False):
    # ===========================
    # BOM PRECEDENCE NETWORK 
    # ===========================
    df = pd.DataFrame({
        "operation": [None] * n,
        "predecessor_operations": [None] * n
    })
    while True:
        p_ij = generate_probabilities(D=D, n=n)
        dict_graph = {}
        g = generate_graph(n, p_ij, dict_graph)
        reversed_graph = {f"J.{i}": [] for i in range(1, len(dict_graph))} # graph reversed to ease next part
        for predecessor, successors in dict_graph.items():
            for successor in successors:
                reversed_graph.setdefault(successor, []).append(predecessor)
        i = 0
        for successor in reversed_graph.keys():
            df.at[i, "operation"] = successor
            df.at[i, "predecessor_operations"] = reversed_graph[successor]
            i += 1
        if not df['operation'].isna().any(): 
            # check if all operations have been included
            # there is a chance that some operations are not included, due to p_ij 
            break
    # Visualization
    if display_graph:
        fig_size = (24, 18) if n >= 30 else (12, 9) if n > 15 else (8, 6) 
        visualize_graph(g, figure_size=fig_size, folder_path=folder_path, file_name=file_name)

    # ===========================
    # OPERATION PROCESSING TIMES
    # ===========================
    log_dist = LogarithmicDistribution(p)
    df['processing_time'] = log_dist.sample(n)

    # ===========================
    # MACHINE ROUTING
    # ===========================
    df = assign_machines(df, M, bottleneck_machine, bottleneck_probability=0.7)

    # ===========================
    # WORKCENTER INFORMATION
    # ===========================
    df['workcenter'] = [f"WC#{random.randint(1, 3)}" for _ in range(n)]

    # ===========================
    # END_PRODUCT &  DUE DATES 
    # ===========================
    def flatten(test_list):
        if isinstance(test_list, list):
            temp = []
            for ele in test_list:
                temp.extend(flatten(ele))
            return temp
        else:
            return [test_list]
    predecessor_operations = flatten(df['predecessor_operations'].tolist())
    unique_operations = list(df['operation'].unique())
    end_products = [item for item in unique_operations if item not in predecessor_operations]
    df['due_date'] = None    
    base_duedate = int(df['processing_time'].sum() / 5)
    for end_product in end_products: 
        df.at[int(end_product[2:])-1, 'due_date'] = base_duedate + random.randint(1,5)
        df.at[int(end_product[2:])-1, 'end_product'] = 1
        # the above code assumes each operation is named "J.xxx" where xxx is the operation number 


    return df 


In [8]:
# def flatten(test_list):
#     if isinstance(test_list, list):
#         temp = []
#         for ele in test_list:
#             temp.extend(flatten(ele))
#         return temp
#     else:
#         return [test_list]

# mylist = flatten(df['predecessors'].tolist())
# unique_operations = list(df['operation'].unique())
# end_products = [item for item in unique_operations if item not in mylist]
# # print(mylist)
# # print(unique_operations)
# print(end_products)


# df['due_date'] = None
# for end_product in end_products: 
#     df.at[int(end_product)-1, 'due_date'] = random.randint(80,100)

# # display(df.head())
# display(df)

In [13]:
# Sample Implementation
random.seed(42)
D = 0.9
n = 100

p_ij = generate_probabilities(D=D, n=n)
df = generate_dataset(
    n=n, 
    M=20, 
    D=D, 
    p=0.25,
    dd=(80,100), 
    display_graph=False
)

display(df)
print(df['operation'].isna().any())

# import matplotlib.pyplot as plt 
# plt.figure(figsize=(10, 6))
# plt.hist(df['processing_time'], edgecolor='black', bins=100)
# plt.title('Histogram of Variable')
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.grid(True)
# plt.show()


# mylist = df['processing_time'].unique().tolist()
# mylist.sort()
# print(mylist)
# for time in mylist: 
#     print(time, len(df.loc[(df['processing_time']==time)]))

,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],1,M6,WC1,None,0
1,J.2,[],1,M3,WC1,None,0
2,J.3,[],1,M12,WC1,None,0
3,J.4,[J.1],1,M12,WC3,None,0
4,J.5,"[J.2, J.3]",1,M4,WC2,None,0
...,...,...,...,...,...,...,...
95,J.96,[],1,M5,WC1,None,0
96,J.97,"[J.94, J.95]",1,M19,WC2,None,0
97,J.98,[J.96],1,M10,WC3,None,0
98,J.99,[],1,M1,WC1,86,1


False


In [31]:
list_n = [200,100,50,30,15]
list_M = [1, 2, 5]
list_D = [0.1, 0.5, 0.9]
list_p = [0.15, 0.95]

combinations = []
for n in list_n: 
    for M in list_M:
        for D in list_D: 
            for p in list_p: 
                combinations.append( (n, M, D, p) )

for n, M, D, p in combinations: 
    folder_path = f"TestCases//{n}operations"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    df = generate_dataset(n, M, D, p, bottleneck_machine=None)
    display(df)
    filename = f"{n}_{M}_{D}_{p}"
    df.to_csv(f"{folder_path}//{filename}.csv", index=False)

    if M > 1:
        df = generate_dataset(n, M, D, p, bottleneck_machine=random.randint(1, M))
        display(df)
        filename_bottleneck = f"{n}_{M}_{D}_{p}_bottleneck"
        df.to_csv(f"{folder_path}//{filename_bottleneck}.csv", index=False)

,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],1,M1,WC#3,None,NaN
1,J.2,[],1,M1,WC#1,None,NaN
2,J.3,"[J.1, J.2]",1,M1,WC#2,None,NaN
3,J.4,[J.3],1,M1,WC#3,None,NaN
4,J.5,[J.4],1,M1,WC#3,None,NaN
...,...,...,...,...,...,...,...
195,J.196,[],1,M1,WC#2,None,NaN
196,J.197,"[J.195, J.196]",2,M1,WC#1,None,NaN
197,J.198,[J.197],1,M1,WC#3,None,NaN
198,J.199,[J.198],1,M1,WC#3,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],6,M1,WC#1,None,NaN
1,J.2,[J.1],2,M1,WC#2,None,NaN
2,J.3,[],2,M1,WC#1,None,NaN
3,J.4,"[J.2, J.3]",1,M1,WC#1,None,NaN
4,J.5,[J.4],42,M1,WC#3,None,NaN
...,...,...,...,...,...,...,...
195,J.196,[J.195],1,M1,WC#3,None,NaN
196,J.197,[J.196],7,M1,WC#1,None,NaN
197,J.198,[J.197],21,M1,WC#1,None,NaN
198,J.199,[J.198],1,M1,WC#1,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],1,M1,WC#1,None,NaN
1,J.2,[],1,M1,WC#1,None,NaN
2,J.3,"[J.1, J.2]",1,M1,WC#3,None,NaN
3,J.4,[],2,M1,WC#2,None,NaN
4,J.5,[J.3],1,M1,WC#2,None,NaN
...,...,...,...,...,...,...,...
195,J.196,[J.195],1,M1,WC#3,None,NaN
196,J.197,[J.196],1,M1,WC#3,None,NaN
197,J.198,[J.197],1,M1,WC#3,None,NaN
198,J.199,[J.198],1,M1,WC#3,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],2,M1,WC#2,None,NaN
1,J.2,[J.1],7,M1,WC#1,None,NaN
2,J.3,[J.2],26,M1,WC#2,None,NaN
3,J.4,[J.3],45,M1,WC#1,None,NaN
4,J.5,[J.4],10,M1,WC#1,None,NaN
...,...,...,...,...,...,...,...
195,J.196,[],1,M1,WC#1,None,NaN
196,J.197,[],1,M1,WC#1,None,NaN
197,J.198,[J.195],14,M1,WC#3,None,NaN
198,J.199,[J.197],1,M1,WC#3,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],2,M1,WC#2,None,NaN
1,J.2,[],1,M1,WC#2,None,NaN
2,J.3,[J.1],1,M1,WC#1,None,NaN
3,J.4,[J.2],1,M1,WC#1,None,NaN
4,J.5,[],1,M1,WC#2,None,NaN
...,...,...,...,...,...,...,...
195,J.196,[J.194],1,M1,WC#1,None,NaN
196,J.197,[J.195],1,M1,WC#1,None,NaN
197,J.198,[J.196],1,M1,WC#1,None,NaN
198,J.199,[],1,M1,WC#2,48,1.0


...

,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],2,M2,WC#3,None,NaN
1,J.2,[J.1],1,M2,WC#1,None,NaN
2,J.3,[J.2],1,M2,WC#1,None,NaN
3,J.4,[],4,M2,WC#3,None,NaN
4,J.5,[J.3],3,M2,WC#3,None,NaN
5,J.6,[J.4],8,M2,WC#3,None,NaN
6,J.7,[],6,M2,WC#2,None,NaN
7,J.8,"[J.5, J.7]",19,M2,WC#1,None,NaN
8,J.9,"[J.6, J.8]",31,M2,WC#2,None,NaN
9,J.10,[J.9],16,M2,WC#2,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],1,M1,WC#1,None,NaN
1,J.2,[],1,M3,WC#1,None,NaN
2,J.3,[J.1],1,M1,WC#1,None,NaN
3,J.4,[J.3],1,M5,WC#2,None,NaN
4,J.5,[J.2],1,M1,WC#2,None,NaN
5,J.6,[],1,M1,WC#2,None,NaN
6,J.7,"[J.4, J.5]",2,M2,WC#2,None,NaN
7,J.8,[],2,M2,WC#3,None,NaN
8,J.9,"[J.6, J.7]",2,M2,WC#1,None,NaN
9,J.10,[J.8],1,M2,WC#2,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],1,M5,WC#3,None,NaN
1,J.2,[],1,M5,WC#1,None,NaN
2,J.3,[],1,M4,WC#2,None,NaN
3,J.4,"[J.1, J.2]",1,M1,WC#1,None,NaN
4,J.5,[J.3],2,M5,WC#2,None,NaN
5,J.6,"[J.4, J.5]",1,M5,WC#2,None,NaN
6,J.7,[],1,M5,WC#1,None,NaN
7,J.8,[],1,M5,WC#3,None,NaN
8,J.9,"[J.6, J.7]",1,M4,WC#3,None,NaN
9,J.10,[],1,M3,WC#2,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],1,M5,WC#3,None,NaN
1,J.2,[],1,M5,WC#3,None,NaN
2,J.3,[J.1],3,M1,WC#3,None,NaN
3,J.4,[],1,M5,WC#1,None,NaN
4,J.5,[J.2],2,M4,WC#1,None,NaN
5,J.6,"[J.3, J.4]",1,M5,WC#3,None,NaN
6,J.7,[],2,M3,WC#1,None,NaN
7,J.8,[J.5],1,M1,WC#3,None,NaN
8,J.9,[J.6],1,M2,WC#3,None,NaN
9,J.10,"[J.7, J.8]",5,M1,WC#3,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],2,M5,WC#3,None,NaN
1,J.2,[],10,M5,WC#3,None,NaN
2,J.3,[J.1],2,M5,WC#1,None,NaN
3,J.4,[J.2],12,M5,WC#3,None,NaN
4,J.5,[J.3],6,M5,WC#2,None,NaN
5,J.6,"[J.4, J.5]",2,M1,WC#3,None,NaN
6,J.7,[J.6],1,M5,WC#1,None,NaN
7,J.8,[],1,M5,WC#3,None,NaN
8,J.9,[],2,M5,WC#1,None,NaN
9,J.10,"[J.7, J.8]",1,M3,WC#2,None,NaN


In [16]:
list_n = [500]
list_M = [5]
list_D = [0.1]
list_p = [0.15, 0.95]

combinations = []
for n in list_n: 
    for M in list_M:
        for D in list_D: 
            for p in list_p: 
                combinations.append( (n, M, D, p) )

for n, M, D, p in combinations: 
    folder_path = f"TestCases//{n}operations"
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    df = generate_dataset(n, M, D, p, bottleneck_machine=None)
    display(df)
    filename = f"{n}_{M}_{D}_{p}"
    df.to_csv(f"{folder_path}//{filename}.csv", index=False)

    if M > 1:
        df = generate_dataset(n, M, D, p, bottleneck_machine=random.randint(1, M))
        display(df)
        filename_bottleneck = f"{n}_{M}_{D}_{p}_bottleneck"
        df.to_csv(f"{folder_path}//{filename_bottleneck}.csv", index=False)

,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],2,M1,WC#2,None,NaN
1,J.2,[],1,M5,WC#2,None,NaN
2,J.3,"[J.1, J.2]",1,M5,WC#1,None,NaN
3,J.4,[J.3],1,M5,WC#1,None,NaN
4,J.5,[J.4],1,M3,WC#3,None,NaN
...,...,...,...,...,...,...,...
495,J.496,[J.495],1,M1,WC#2,None,NaN
496,J.497,[J.496],2,M5,WC#3,None,NaN
497,J.498,[J.497],1,M1,WC#1,None,NaN
498,J.499,[J.498],1,M4,WC#2,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],1,M1,WC#2,None,NaN
1,J.2,[J.1],1,M5,WC#3,None,NaN
2,J.3,[J.2],1,M3,WC#2,None,NaN
3,J.4,[J.3],1,M5,WC#2,None,NaN
4,J.5,[J.4],1,M2,WC#1,None,NaN
...,...,...,...,...,...,...,...
495,J.496,[J.495],1,M3,WC#3,None,NaN
496,J.497,[J.496],1,M5,WC#2,None,NaN
497,J.498,[J.497],1,M1,WC#2,None,NaN
498,J.499,[J.498],1,M5,WC#3,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],3,M5,WC#3,None,NaN
1,J.2,[J.1],1,M1,WC#2,None,NaN
2,J.3,[J.2],2,M5,WC#3,None,NaN
3,J.4,[J.3],3,M5,WC#2,None,NaN
4,J.5,[J.4],5,M2,WC#1,None,NaN
...,...,...,...,...,...,...,...
495,J.496,[J.495],18,M5,WC#2,None,NaN
496,J.497,[J.496],5,M2,WC#3,None,NaN
497,J.498,[J.497],8,M2,WC#3,None,NaN
498,J.499,[],2,M5,WC#2,None,NaN


,operation,predecessor_operations,processing_time,machine,workcenter,due_date,end_product
0,J.1,[],1,M2,WC#3,None,NaN
1,J.2,[J.1],11,M3,WC#2,None,NaN
2,J.3,[J.2],14,M3,WC#1,None,NaN
3,J.4,[J.3],1,M3,WC#2,None,NaN
4,J.5,[J.4],3,M3,WC#1,None,NaN
...,...,...,...,...,...,...,...
495,J.496,[J.495],1,M3,WC#1,None,NaN
496,J.497,[J.496],6,M3,WC#2,None,NaN
497,J.498,[J.497],4,M4,WC#2,None,NaN
498,J.499,[J.498],2,M2,WC#1,None,NaN
